# Chapter 14: The Collections Module

Python's `collections` module provides specialized container data types that go beyond the built-in `list`, `dict`, `tuple`, and `set`. These containers are optimized for specific use cases and can make your code both cleaner and more efficient.

## Key Types Covered
- **Counter**: Count hashable elements
- **defaultdict**: Dicts with default values for missing keys
- **deque**: Double-ended queue for efficient append/pop from both ends
- **ChainMap**: Layered dictionary lookup
- **OrderedDict**: Insertion-ordered dict (historical)
- **namedtuple**: Tuple subclasses with named fields

## Counter: Counting Hashable Elements

`Counter` is a `dict` subclass for counting hashable objects. It maps elements to their counts, making frequency analysis trivial.

In [ ]:
from collections import Counter

# Count characters in a string
letter_counts = Counter("abracadabra")
print(f"Letter counts: {letter_counts}")
print(f"Count of 'a': {letter_counts['a']}")
print(f"Count of 'b': {letter_counts['b']}")
print(f"Count of 'z' (missing): {letter_counts['z']}")

# most_common() returns elements sorted by frequency
print(f"\nTop 2 most common: {letter_counts.most_common(2)}")

# Count words in a list
words = ["apple", "banana", "apple", "cherry", "banana", "apple"]
word_counts = Counter(words)
print(f"\nWord counts: {word_counts}")
print(f"Most common word: {word_counts.most_common(1)[0]}")

In [ ]:
# Counter arithmetic operations
inventory_a = Counter(apples=3, oranges=2, bananas=1)
inventory_b = Counter(apples=1, oranges=4, grapes=5)

# Addition combines counts
combined = inventory_a + inventory_b
print(f"Combined inventory: {combined}")

# Subtraction removes counts (drops zero and negative)
difference = inventory_a - inventory_b
print(f"A - B (positive only): {difference}")

# Intersection (minimum of counts)
common = inventory_a & inventory_b
print(f"Intersection (min): {common}")

# Union (maximum of counts)
union = inventory_a | inventory_b
print(f"Union (max): {union}")

# total() gives the sum of all counts
print(f"\nTotal items in A: {inventory_a.total()}")

## defaultdict: Automatic Default Values

`defaultdict` is a `dict` subclass that calls a factory function to supply missing values. This eliminates the need for `setdefault()` or key-existence checks.

In [ ]:
from collections import defaultdict

# Using list as default factory - group items by category
groceries: defaultdict[str, list[str]] = defaultdict(list)
groceries["fruit"].append("apple")
groceries["fruit"].append("banana")
groceries["vegetable"].append("carrot")
groceries["dairy"].append("milk")

print("Grouped groceries:")
for category, items in groceries.items():
    print(f"  {category}: {items}")

# Accessing a missing key creates it with the default
print(f"\nMeat (missing key, auto-created): {groceries['meat']}")
print(f"'meat' now in dict: {'meat' in groceries}")

In [ ]:
# Using int as default factory - counting occurrences
word_count: defaultdict[str, int] = defaultdict(int)
sentence = "the cat sat on the mat and the cat slept"
for word in sentence.split():
    word_count[word] += 1  # int() returns 0 by default

print("Word frequencies:")
for word, count in sorted(word_count.items(), key=lambda x: -x[1]):
    print(f"  '{word}': {count}")

# Using set as default factory - track unique values
student_courses: defaultdict[str, set[str]] = defaultdict(set)
enrollments = [
    ("Alice", "Math"), ("Bob", "Science"),
    ("Alice", "Science"), ("Alice", "Math"),  # duplicate
]
for student, course in enrollments:
    student_courses[student].add(course)

print("\nStudent courses (duplicates removed):")
for student, courses in student_courses.items():
    print(f"  {student}: {courses}")

## deque: Double-Ended Queue

`deque` (pronounced "deck") provides O(1) append and pop operations from both ends, compared to O(n) for `list.insert(0, x)` or `list.pop(0)`.

In [ ]:
from collections import deque

# Basic deque operations
d: deque[int] = deque([1, 2, 3])
print(f"Initial deque: {d}")

# Append to both ends
d.appendleft(0)
d.append(4)
print(f"After appendleft(0), append(4): {d}")

# Pop from both ends
left = d.popleft()
right = d.pop()
print(f"Popped left={left}, right={right}: {d}")

# Extend from both ends
d.extendleft([10, 20])  # Note: items are added one at a time, reversing order
d.extend([30, 40])
print(f"After extendleft and extend: {d}")

# Rotate elements
d_rotate = deque([1, 2, 3, 4, 5])
d_rotate.rotate(2)   # Rotate right by 2
print(f"\nRotate right by 2: {d_rotate}")
d_rotate.rotate(-2)  # Rotate left by 2
print(f"Rotate left by 2:  {d_rotate}")

In [ ]:
# Bounded deque with maxlen - keeps only the most recent items
recent_logs: deque[str] = deque(maxlen=3)
logs = ["startup", "request_1", "request_2", "request_3", "shutdown"]

for log in logs:
    recent_logs.append(log)
    print(f"Added '{log}': {list(recent_logs)}")

print(f"\nFinal (only last 3): {list(recent_logs)}")

# Practical example: sliding window of recent values
def moving_average(values: list[float], window: int) -> list[float]:
    """Calculate a moving average using a bounded deque."""
    d: deque[float] = deque(maxlen=window)
    averages: list[float] = []
    for val in values:
        d.append(val)
        averages.append(sum(d) / len(d))
    return averages

data = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0]
print(f"\nData: {data}")
print(f"Moving avg (window=3): {moving_average(data, 3)}")

## ChainMap: Layered Dictionary Lookup

`ChainMap` groups multiple dictionaries into a single view. Lookups search through the dicts in order, returning the first match. This is perfect for layered configurations (e.g., command-line > environment > defaults).

In [ ]:
from collections import ChainMap

# Layered configuration: overrides take precedence over defaults
defaults = {"color": "blue", "size": "medium", "font": "Arial"}
user_prefs = {"color": "red", "font": "Helvetica"}
cli_args = {"color": "green"}

# First dict has highest priority
config = ChainMap(cli_args, user_prefs, defaults)

print(f"color: {config['color']}")  # From cli_args
print(f"font:  {config['font']}")   # From user_prefs
print(f"size:  {config['size']}")   # From defaults

# See all keys across all maps
print(f"\nAll keys: {list(config.keys())}")
print(f"Maps: {config.maps}")

# new_child() creates a new layer on top
session_overrides = config.new_child({"size": "large"})
print(f"\nWith session override - size: {session_overrides['size']}")
print(f"Original config - size: {config['size']}")

## OrderedDict: Insertion-Ordered Dictionary

Before Python 3.7, regular `dict` did not guarantee insertion order. `OrderedDict` was created to fill that gap. Since Python 3.7+, regular dicts maintain insertion order, so `OrderedDict` is less essential. However, it still offers `move_to_end()` and order-sensitive equality.

In [ ]:
from collections import OrderedDict

# OrderedDict preserves insertion order (same as dict in 3.7+)
od = OrderedDict()
od["first"] = 1
od["second"] = 2
od["third"] = 3
print(f"OrderedDict: {od}")

# move_to_end() - unique to OrderedDict
od.move_to_end("first")  # Move to the end
print(f"After move 'first' to end: {od}")

od.move_to_end("third", last=False)  # Move to the beginning
print(f"After move 'third' to start: {od}")

# Order matters for equality in OrderedDict
od1 = OrderedDict(a=1, b=2)
od2 = OrderedDict(b=2, a=1)
print(f"\nOrderedDict order-sensitive equality: {od1 == od2}")

# Regular dicts ignore order for equality
d1 = {"a": 1, "b": 2}
d2 = {"b": 2, "a": 1}
print(f"Regular dict equality: {d1 == d2}")

## namedtuple: Tuples with Named Fields

`namedtuple` creates tuple subclasses with named fields, making code more readable. They are lightweight, immutable, and support both indexing and attribute access.

In [ ]:
from collections import namedtuple

# Create a namedtuple type
Point = namedtuple("Point", ["x", "y"])
p = Point(3, 4)

# Access by name or index
print(f"Point: {p}")
print(f"By name: x={p.x}, y={p.y}")
print(f"By index: p[0]={p[0]}, p[1]={p[1]}")

# Immutable - cannot reassign fields
try:
    p.x = 10
except AttributeError as e:
    print(f"\nCannot modify: {e}")

# _replace creates a new instance with modified fields
p2 = p._replace(x=10)
print(f"Original: {p}, Replaced: {p2}")

# Practical example: representing records
Employee = namedtuple("Employee", "name department salary")
emp = Employee("Alice", "Engineering", 95000)
print(f"\nEmployee: {emp}")
print(f"Fields: {emp._fields}")
print(f"As dict: {emp._asdict()}")

## Practical Example: Log Analysis Pipeline

Let's combine several `collections` types to build a small log analysis pipeline.

In [ ]:
from collections import Counter, defaultdict, deque, namedtuple

# Define a structured log entry
LogEntry = namedtuple("LogEntry", "timestamp level message")

# Sample log data
raw_logs = [
    LogEntry("10:01", "INFO", "Server started"),
    LogEntry("10:02", "INFO", "Request from user_1"),
    LogEntry("10:03", "WARNING", "High memory usage"),
    LogEntry("10:04", "ERROR", "Database timeout"),
    LogEntry("10:05", "INFO", "Request from user_2"),
    LogEntry("10:06", "ERROR", "Database timeout"),
    LogEntry("10:07", "INFO", "Request from user_1"),
    LogEntry("10:08", "WARNING", "Disk space low"),
    LogEntry("10:09", "INFO", "Cache cleared"),
    LogEntry("10:10", "ERROR", "Connection refused"),
]

# Count log levels
level_counts = Counter(entry.level for entry in raw_logs)
print("Log level distribution:")
for level, count in level_counts.most_common():
    print(f"  {level}: {count}")

# Group messages by level
by_level: defaultdict[str, list[str]] = defaultdict(list)
for entry in raw_logs:
    by_level[entry.level].append(entry.message)

print("\nError messages:")
for msg in by_level["ERROR"]:
    print(f"  - {msg}")

# Keep only the last 5 log entries (recent history)
recent: deque[LogEntry] = deque(maxlen=5)
for entry in raw_logs:
    recent.append(entry)

print("\nMost recent 5 entries:")
for entry in recent:
    print(f"  [{entry.timestamp}] {entry.level}: {entry.message}")

## Summary

### Key Takeaways

| Type | Use Case | Key Feature |
|------|----------|-------------|
| **Counter** | Frequency counting | `most_common()`, arithmetic ops |
| **defaultdict** | Auto-initialize missing keys | Factory function (list, int, set) |
| **deque** | Fast append/pop both ends | O(1) operations, `maxlen` |
| **ChainMap** | Layered dict lookup | Priority-based config |
| **OrderedDict** | Order-sensitive equality | `move_to_end()` |
| **namedtuple** | Readable, immutable records | Named fields, `_replace()`, `_asdict()` |

### When to Reach for `collections`
- **Counting things?** Use `Counter` instead of manual dict incrementing
- **Grouping items?** Use `defaultdict(list)` instead of `dict.setdefault()`
- **Need a queue or stack?** Use `deque` instead of `list` for O(1) ends
- **Layered config?** Use `ChainMap` for clean precedence chains
- **Lightweight records?** Use `namedtuple` for immutable data carriers